# Data

The original dataset is made from training (15000 samples), dev (6431) and testing (6500) files. Only the 400 *i-vector* features where used. A process to whiten the entire dataset was applied before using the feature set $x_i$ and only the dev set was used to train the whitening parameters (see code suplied with the data by the competition organizers). Each sample is either unlabeled (all dev and testing samples) and we will label it as $y_i=0$ or is one of $n=50$ different categories $y_i \in \{1 \ldots n \}$ (all training samples.)

## Cross validation

In order to select hyper parameters of the model a modified [cross validation dataset was built](./fuel.ipynb) from the training dataset.

In the modified dataset, the $n$ known original training labels are considered to be the entire label space of the modified dataset and from them a subset is assumed to be known.
The other labeles are assumed to be out-of-set for the purpose of the modified dataset.
The number of assumed known labels is such that the ratio of known and unknown labels in the modified set is:

$Q = \lfloor \left( 1 - P_{\text{oos}} \right) * n \rfloor = 38 \quad P_{\text{oos}} = 0.23$

The labels of the modified dataset are re-indexed such that the labels assumed to be known are $y_i \in \{1 \ldots Q \}$

A part, $1-r$, of the training data with labels assumed to be known is used for training as labeled data. The rest, $r$, of the samples with labels assumed to be known are mixed with $r$ of the rest of the training which has labels that are assumed to be out-of-set. The mix is used for training as unlabeled data.
For having an the number of unlabeled samples to be $u=0.5$ from the number of labeled samples (the ratio between `dev` and `training` sizes):


$r = Q*u/(50+Q*u)$

The remaining $1-r$ samples with labels assumed to be unknown are dropped.

Each of the steps above, in building the modified dataset, uses a random selection process. The process of creating a modified dataset can be repeated many times giving each label an opportunity to be out-of-set.

# Model Training

When training a model all samples are used, labeled and unlabeled. For cross validation, this is the modified dataset and for submission this is training and dev datasets, the test dataset is only used to make final prediction for submission.

The model generates probability for each sample, $x_i$, to be out-of-set or in one of the categories. When doing cross validation the model will generate $Q+1=39$ categories and when training on the entire available data the model will generate $n+1=51$ categories. The label $l=0$ is used for out-of-set prediction (not to be confused with unlabeled sample.)

$p(l) = p(l \mid x_i) \quad l \in \{ 0 \ldots Q \} \quad \text{or} \quad l \in \{ 0 \ldots n \} $

## Final score

According to the [evaluation plan](http://www.nist.gov/itl/iad/mig/upload/lre_ivectorchallenge_rel_v2.pdf) of the competition, the goal is to minimize:

$\text{Cost} = \frac{1-P_{\text{oos}}}{n} * \sum_{k=1}^n P_{\text{error}}(k) + P_{\text{oos}} * P_{\text{error}}(\text{oos}) \qquad [1]$

$P_{\text{error}}(k) = \left( \frac{\text{#errors_class_k}}{\text{#trials_class_k}}  \right) $

In the cross validation stage we can compute this cost directly, by replacing $n$ with $Q$, and using the information we have on the validation part of the modified dataset. We will use this score to select the best hyper parameters.

## Loss function

The training process optimize the model internal parameters (weights) minimizing a loss function. We describe the loss function used in cross validation training, when training for a submission, substitute $Q=n$.

The loss is computed as a sum of loss on batches of samples. Each batch has ($N=1024$) samples. For each sample, $x_i$, the loss function accepts as input the $Q+1$ probabilities, $p(l \mid x_i)$ from the model and the label information, $y_i$. Note that $p(0 \mid x_i)$ gives the probability of the model to out-of-set label and $y_i = 0$ is used to indicate that the sample $x_i$ is not labeled.

The loss of a batch is made from several parts:

$\text{loss} = \text{cross_entopy} + \beta \cdot \text{aprior_average_cross_entropy} + \gamma \cdot \text{binary_cross_entropy} \qquad [2]$

where $\beta$ and $\gamma$ are hyper-parameters. After running cross validation tests the values $\beta=0.15$ and $\gamma=0.01$ were selected.

### cross entropy
for the labeled samples in the batch the loss is

$\text{cross_entopy} = \frac{1}{N_l} \sum_{i : y_i \in \{1 \ldots Q \}} -\log p(y_i \mid x_i)$

were $N_l$ is the number of labeled samples in the batch

$N_l = \sum_{i : y_i \in \{1 \ldots Q \}} 1$

### aprior  cross entropy
Aprior, we assume that the predicted probabilities of unlabeled samples should have the distribution:

$P^a (0) = P_\text{oos} \quad P^a (l) = \frac{1-P_\text{oos}}{Q} \quad \forall l \in \{1 \ldots Q \}$

This distribution is correct for the cross validation modified dataset and we assume it is correct for the dev dataset.

Armed with the apriori distribution, we can add a loss term which measure the cross entropy between predictions made on unlabeled samples and this
apriori distribution:

$\text{aprior_cross_entropy} = \frac{1}{N_u} \sum_{i : y_i \notin \{1 \ldots Q \}} -P^a(l)\log(p(l \mid x_i))$

were $N_u$ is the number of labeled samples in the batch

$N_u = \sum_{i : y_i \notin \{1 \ldots Q \}} 1$

### aprior average cross entropy
However it was found that a much better result is achieved by first averaging all the predicted probabilities over the unlabeled samples in the batch and only then 
measuring its cross entropy with the aprior probability:

$\bar{p}(l) = \frac{1}{N_u} \sum_{i : y_i \notin \{1 \ldots Q \}} p(l \mid x_i) \\
\text{aprior_average_cross_entropy} = - \sum_{l=0}^Q P^a (l) \log(\bar{p}(l \mid x_i))$

### aprior average Dirichlet

$C_2 = -􀀀p_\text{oos} \log p_\text{av}(\text{oos}) 􀀀- \frac{1 - p_\text{oos}}{k} \sum_{i=1}^k \log p_\text{av}(i)$

changes to 

$\text{NLLK}(p_\text{av}) = -(\alpha_\text{oos} - 1) \log p_\text{av}(\text{oos}) 􀀀- \sum_{i=1}^k (\alpha_i - 1) \log p_\text{av}(i) \quad + \text{constant}$

such that 

$p_\text{oos} = \frac{\alpha_\text{oos}}{\alpha_\text{sum}} \qquad \frac{1 - p_\text{oos}}{k} = \frac{\alpha_i}{\alpha_\text{sum}}$

where

$\alpha_\text{sum} = \alpha_\text{oos} + \sum_{i=1}^k \alpha_i$

redefine $C_2$ as

$C_2 = -(􀀀p_\text{oos} - \delta) \log p_\text{av}(\text{oos}) 􀀀- \left( \frac{1 - p_\text{oos}}{k} - \delta \right) \sum_{i=1}^k \log p_\text{av}(i)$

where $\alpha_\text{sum}$ is moved outside into $C_2$ scale factor $\alpha$ and $\delta = 1/\alpha_\text{sum}$


### binary cross entropy
We will use $p(0 \mid x_i)$ to predict if $x_i$ is out-of-set or not. If $x_i$ happens to be a labeled sample, we know it is not out-of-set and if it is unlabeled we know there is $P_\text{oos}$ chance that it is out-of-set.
Again this is something which is true for the corss validation modified dataset and assumed to be true for the dev dataset:

$\text{binary_cross_entropy} = -\frac{1}{N} \left[ \sum_{i:y_i \notin \{1 \ldots Q \}} \left( P_\text{oos} \log(p_0(i)) + (1-P_\text{oos}) \log(p_1(i)) \right) + \sum_{i:y_i \in \{1 \ldots Q \}} \log(p_1(i)) \right]$

were

$p_0(i) = p(0 \mid x_i) \quad p_1(i) = 1-p_0(i)$

# Model

The loss function we use [2] is applied to all available data: training and dev datasets. However the strongest signal is from the training (labeled) part and effectively we are in a situation in which 1/3 of the available data is unlabeled. It is therefore beneficial to use semi-supervised technique which will utilize the information available in all the data and not just in the training set.

Predictions, $p(y_i \mid x_i)$, are made using a modified [Ladder Network](http://arxiv.org/abs/1507.02672). The original Ladder Network [code](https://github.com/CuriousAI/ladder) was slightly modified. The code was modified to accept the training and dev data of the competition and was used in its entire both for supervised and unsupervised parts of the ladder method. The objective function used in computing the cost of the supervised part of the ladder method was replaced from a simple Cross Entropy to the loss function [2]. In addition, the error rate [1] was monitored while training on cross-validation dataset to determine the optimal number of epochs for training. The setup used for training that gave the best results are as follows:

```bash
python run.py train --lr 1e-3 --labeled-samples 21431 --unlabeled-samples 21431 --encoder-layers 500-500-500-100-51 --decoder-spec gauss,relu,relu,relu,relu,relu --denoising-cost-x 1,1,.3,.3,.3,.3 --dseed 0 --seed 2 --super-noise-std 0.5 --f-local-noise-std 0.5 --batch-size 1024 --valid-batch-size 1024 --num-epochs 1000 --dataset 160111-fuel.test -- test.
```

The interpretation of each of the parameters is as follows:

parameter | value | description
--- | --- | ---
dataset | 160111-fuel.test | Both training and dev datasets were used as input. For cross validation this was changed to `160111-fuel.train`
labeled-samples | 21431 | All samples in training and dev were used for training the supervised part of the ladder method. This is made possible because the modified loss function has a part which is applied on unlabeled samples. For cross validation this was modified to `10000` and the rest of the modified dataset was used for validation
unlabeled-samples | 21431 | All samples in training and dev were used in the unsupervised parts of the ladder method. For cross validation this was modified to `10000`
encoder-layers | 500-500-500-100-51 | The network has an input of dimension 400 which pass through 4 hidden layers of size 500, 500, 500 and 100 and a final output layer of 51. For cross validation this was modified to 39.
decoder-spec | gauss,relu,relu,relu,relu,relu | A direct skip of information from the encoder to the decoder was used only on the input layer using the gaussian method described in ladder paper.
denoising-cost-x | 1,1,.3,.3,.3,.3 | The L2 error of the de-noising layers compared with an un-noised clean encoder was weighted with a weight of 1 for the input layer and the first hidden layer and 0.3 for all other layers.
super-noise-std | 0.5 | std of gaussian noise added to the input of the courrputed encoder
f-local-noise-std | 0.5 | std of gaussian noise added to output of all layers courrputed encoder
lr | 1e-3 | Learning rate
num-epochs | 1000 | Number of epoch iterations for which training was made. Before each iteration the order of the samples was shuffled. It turns out that because of the unsupervised learning the ladder method is insensitive to the number of epochs and having between 800 to 2000 epoch iterations would give similar results
batch-size  | 1024| batch size used for training. this size has a secondary effect through the loss function which performed an average of predictions before computing the loss
lrate-decay | 0.67 (default) | the learning rate starts to decay linearly to zero after passing 0.67 of the epoch iterations
act | relu (default) | the activation of the encoder layers except for the last layer which is always softmax

# Results

the results where measured by generating predictions on the test dataset using the model found in the training process. The prediction were then submitted to the competition web site which used an unknown subset of 30% of the samples to compute a score for the PROGESS SET (results for the 70% eval set are not reported by the web site.) 

Score | Description | Command line
--- | --- | ---
24.000 | The best configuration which was described above. This would have been translated to 11th place while the competition was in progress | --lr 1e-3 --labeled-samples 21431 --unlabeled-samples 21431 --encoder-layers 500-500-500-100-51 --decoder-spec gauss,relu,relu,relu,relu,relu --denoising-cost-x 1,1,.3,.3,.3,.3 --dseed 0 --seed 2 --super-noise-std 0.5 --f-local-noise-std 0.5 --batch-size 1024 --valid-batch-size 1024 --num-epochs 1000
31.487 | In this configuration the unsupervisied part of the ladder algorithm is disabled. An early stopping after 138 epochs was needed to avoid overfiting | --lr 1e-3 --labeled-samples 21431 --unlabeled-samples 21431 --encoder-layers 500-500-500-100-51 --decoder-spec gauss,relu,relu,relu,relu,relu --denoising-cost-x 0,0,0,0,0,0 --decoder-spec 0-0-0-0-0-0 --dseed 0 --seed 2 --super-noise-std 0.5 --f-local-noise-std 0.5 --batch-size 1024 --valid-batch-size 1024 --num-epochs 138

To generate a submission file identify the directory in which the training stored its results. This is a subdirectory under `./results/` the subdirectory name has prefix determined by the last argument in the command line. In the example given above the prefix is `test.`. The suffix of the subdirectory is a number which is incremented after every training run. Below I assume that all of this results in `results/test.0`

You then generate predictions with
```bash
run.py dump --layer -1 -- results/test.0
```

The submission is made from the predictions on the `test` part of the dataset file (last 6500 samples) that are saved in `bz2` file which can be submitted to the web site

In [ ]:
import numpy as np
import bz2

yprob = np.load('results/test.0/layer-1.npy'%t)
y_pred = np.argmax(yprob,axis=1)
fn = 'submission.txt.bz2'
with bz2.BZ2File('data/%s'%fn, 'w') as f:
    for i in y_pred[-6500:]:
        f.write('%s\n' % idx2lang[i])

# Reference

1. https://ivectorchallenge.nist.gov/
2. http://www.nist.gov/itl/iad/mig/upload/lre_ivectorchallenge_rel_v2.pdf
2. http://arxiv.org/abs/1507.02672
3. https://github.com/CuriousAI/ladder
3. http://arxiv.org/abs/1511.06430v3